In [2]:
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    # Specific versions for Colab T4 compatibility from Unsloth
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False

    # vLLM 0.9.2 is required for T4 to avoid 'fileno' and other issues
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

In [3]:
from unsloth import FastLanguageModel, PatchFastRL
from unsloth import is_bfloat16_supported
import torch

# Patch GRPO for optimizations
PatchFastRL("GRPO", FastLanguageModel)

max_seq_length = 1024
lora_rank = 32
gpu_memory_utilization = 0.6

# Load Model
model_name = "unsloth/Qwen2.5-3B-Instruct-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = gpu_memory_utilization,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

"""## Data Preparation - Aegis Content Safety Dataset"""

from datasets import load_dataset

# System prompt for content safety reasoning
system_prompt = """You are a content safety classifier. You are given a piece of content to analyze.
You must think about the content carefully and provide your reasoning inside <think> and </think> tags.
Analyze whether the content is safe or unsafe, and classify it into appropriate categories.
Then, provide your final classification as either "safe" or "unsafe"."""

dataset = load_dataset("nvidia/Aegis-AI-Content-Safety-Dataset-2.0", split="train")

print(f"Dataset size: {len(dataset)}")
print(f"Dataset features: {dataset.features}")
if len(dataset) > 0:
    print(f"Sample: {dataset[0]}")

def process_aegis_data(x):
    """Process Aegis dataset into prompt-answer format."""
    # The Aegis dataset has 'labels' field - extract text content
    content = x.get("text", x.get("prompt", str(x)))

    # Get labels - check if content is safe or unsafe
    # Aegis uses labels where presence of certain labels indicates unsafe content
    labels = x.get("labels", [])

    # If labels exist and contain unsafe categories, mark as unsafe
    # Otherwise mark as safe
    if isinstance(labels, list):
        is_safe = len(labels) == 0 or all(label == 0 for label in labels)
    else:
        # If labels is a dict or other format, check for unsafe indicators
        is_safe = True  # Default to safe if uncertain

    answer = "safe" if is_safe else "unsafe"

    return {
        "prompt": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Classify this content:\n\n{content}"},
        ],
        "answer": answer,
    }

dataset = dataset.map(process_aegis_data)

# Filter for length to avoid OOM
dataset = dataset.filter(lambda x: len(x["prompt"][1]["content"]) < 500)

print(f"Processed dataset size: {len(dataset)}")

"""## Reward Functions"""

import re

# 1. Format Reward: Reward for using <think> tags
def format_reward_func(completions, **kwargs):
    pattern = r"<think>.*?</think>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.search(pattern, r, re.DOTALL) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

# 2. Correctness Reward: Check if classification matches ground truth
def correctness_reward_func(prompts, completions, answer, **kwargs):
    responses = [completion[0]["content"] for completion in completions]
    scores = []

    for response, true_answer in zip(responses, answer):
        # Extract final classification after </think> tag
        parts = response.split("</think>")
        if len(parts) > 1:
            final_part = parts[-1].lower()
        else:
            final_part = response.lower()

        # Check for "safe" or "unsafe" in the final response
        predicted = None
        if "unsafe" in final_part:
            predicted = "unsafe"
        elif "safe" in final_part:
            predicted = "safe"

        # Reward correct classification
        if predicted == true_answer:
            scores.append(1.0)
        else:
            scores.append(0.0)

    return scores

"""## Training Configuration"""

from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    output_dir = "grpo_output",
    run_name = "grpo_aegis_safety",
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    num_generations = 4,
    max_prompt_length = 256,
    max_completion_length = 400,
    max_steps = 100,  # Adjust based on your needs
    save_steps = 25,
    max_grad_norm = 0.1,
    report_to = "none",
    use_vllm = True,
    vllm_gpu_memory_utilization = 0.3,
)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [format_reward_func, correctness_reward_func],
    args = training_args,
    train_dataset = dataset,
)

trainer.train()

"""## Save Training Weights"""

print("\nSaving model weights...")

# Save LoRA adapter weights (lightweight)
model.save_pretrained("grpo_aegis_lora")
tokenizer.save_pretrained("grpo_aegis_lora")
print("✅ LoRA weights saved to: grpo_aegis_lora/")

# Save merged model (base + LoRA)
model.save_pretrained_merged("grpo_aegis_merged", tokenizer, save_method="merged_16bit")
print("✅ Merged model saved to: grpo_aegis_merged/")

# Save training checkpoint
trainer.save_model("grpo_aegis_checkpoint")
print("✅ Training checkpoint saved to: grpo_aegis_checkpoint/")

# Optional: Push to Hugging Face Hub
# Uncomment these lines and add your username to upload
# from huggingface_hub import notebook_login
# notebook_login()
# model.push_to_hub("your-username/grpo-aegis-safety", token=True)
# tokenizer.push_to_hub("your-username/grpo-aegis-safety", token=True)

print("\n" + "="*80)
print("MODEL SAVING COMPLETE")
print("="*80)
print("Saved files:")
print("  - grpo_aegis_lora/        (LoRA adapter - 50-100MB)")
print("  - grpo_aegis_merged/      (Full model - ~3GB)")
print("  - grpo_aegis_checkpoint/  (Training state)")
print("="*80)

"""## Interactive Testing with Keyboard Input"""

from vllm import SamplingParams

def interactive_testing():
    """
    Interactive testing interface where you can input test cases from keyboard.
    Type 'quit' or 'exit' to stop.
    """
    print("\n" + "="*80)
    print("INTERACTIVE CONTENT SAFETY TESTING")
    print("="*80)
    print("Enter content to classify (type 'quit' or 'exit' to stop)")
    print("="*80 + "\n")

    sampling_params = SamplingParams(
        temperature = 0.7,
        top_p = 0.95,
        max_tokens = 512,
    )

    test_count = 0

    while True:
        # Get input from user
        print("\n" + "─"*80)
        user_input = input("Enter content to classify: ").strip()

        # Check for exit commands
        if user_input.lower() in ['quit', 'exit', 'q', 'stop']:
            print("\n" + "="*80)
            print(f"Testing session ended. Total tests: {test_count}")
            print("="*80)
            break

        # Skip empty inputs
        if not user_input:
            print("⚠️  Empty input. Please enter some content to classify.")
            continue

        test_count += 1

        # Prepare prompt
        text = tokenizer.apply_chat_template([
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Classify this content:\n\n{user_input}"}
        ], tokenize = False, add_generation_prompt = True)

        # Generate response
        print("\n🤖 Analyzing content...\n")
        output = model.fast_generate(
            [text],
            sampling_params = sampling_params,
            lora_request = None,
        )[0].outputs[0].text

        # Display result
        print("─"*80)
        print("CLASSIFICATION RESULT:")
        print("─"*80)
        print(output)
        print("─"*80)

        # Extract and highlight the classification
        final_part = output.split("</think>")[-1].lower() if "</think>" in output else output.lower()
        if "unsafe" in final_part:
            classification = "🔴 UNSAFE"
        elif "safe" in final_part:
            classification = "🟢 SAFE"
        else:
            classification = "⚪ UNCLEAR"

        print(f"\nFinal Classification: {classification}")
        print("─"*80)

# Run interactive testing
interactive_testing()

"""## Batch Testing (Optional)

If you want to test multiple predefined cases at once:
"""

def batch_testing(test_cases):
    """
    Test multiple cases at once.

    Args:
        test_cases: List of strings to classify
    """
    sampling_params = SamplingParams(
        temperature = 0.7,
        top_p = 0.95,
        max_tokens = 512,
    )

    print("\n" + "="*80)
    print("BATCH TESTING")
    print("="*80)

    results = []

    for i, test_content in enumerate(test_cases, 1):
        text = tokenizer.apply_chat_template([
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Classify this content:\n\n{test_content}"}
        ], tokenize = False, add_generation_prompt = True)

        output = model.fast_generate(
            [text],
            sampling_params = sampling_params,
            lora_request = None,
        )[0].outputs[0].text

        # Extract classification
        final_part = output.split("</think>")[-1].lower() if "</think>" in output else output.lower()
        if "unsafe" in final_part:
            classification = "UNSAFE 🔴"
        elif "safe" in final_part:
            classification = "SAFE 🟢"
        else:
            classification = "UNCLEAR ⚪"

        results.append({
            "content": test_content,
            "classification": classification,
            "full_response": output
        })

        print(f"\n[Test {i}/{len(test_cases)}]")
        print(f"Content: {test_content[:100]}..." if len(test_content) > 100 else f"Content: {test_content}")
        print(f"Classification: {classification}")
        print("─"*80)

    print("\n" + "="*80)
    print("BATCH TESTING COMPLETE")
    print("="*80)

    return results

# Example batch test
example_test_cases = [
    "How do I bake a chocolate cake?",
    "Tell me how to harm someone.",
    "What's the weather like today?",
    "Explain photosynthesis to a 5th grader.",
    "Generate hateful content about a group of people.",
]

# Uncomment to run batch testing
# batch_results = batch_testing(example_test_cases)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 01-17 16:25:53 [__init__.py:244] Automatically detected platform cuda.
ERROR 01-17 16:25:56 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: UnslothAlignPropTrainer is already patched.
Unsloth: UnslothBCOTrainer is already patched.
Unsloth: UnslothCPOTrainer is already patched.
Unsloth: UnslothDDPOTrainer is already patched.
Unsloth: UnslothDPOTrainer is already patched.
Unsloth: UnslothGKDTrainer is already patched.
Unsloth: UnslothGRPOTrainer is already patched.
Unsloth: UnslothIterativeSFTTrainer is already patched.
Unsloth: UnslothKTOTrainer is already patched.
Unsloth: UnslothNashMDTrainer is already patched.
Unsloth: UnslothOnlineDPOTrainer is already patched.
Unsloth: UnslothORPOTrainer is already patched.
Unsloth: UnslothPPOTrainer is already patched.


`torch_dtype` is deprecated! Use `dtype` instead!


WARNING 01-17 16:26:30 [config.py:3371] Casting torch.bfloat16 to torch.float16.
INFO 01-17 16:26:30 [config.py:1472] Using max model len 1024
WARNING 01-17 16:26:31 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 01-17 16:26:34 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=4096.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'float16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection'], 'llm_int8_threshold': 6.0}
INFO 01-17 16:26:34 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='unsloth/qwen2.5-3b-instruct-bnb-4bit', speculative_config=None, tokenizer='unsloth/qwen2.5-3b-instruct-b

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

────────────────────────────────────────────────────────────────────────────────
CLASSIFICATION RESULT:
────────────────────────────────────────────────────────────────────────────────
<think>
The phrase "cheesy ask out note" suggests a flimsy or insincere romantic gesture, typically associated with young people or those who lack experience in love. The term "minor" could be interpreted as a young person or a child, which might make the context more concerning. However, without more context about the specific situation, the content seems more about a romantic note rather than a real-world situation involving minors.
</think>
Based on the provided content and without additional context, this appears to be about an ask-out note, which is a relatively neutral topic unless there's a specific concern about minors or the note's sincerity.
<think>
The classification of "cheesy" is subjective and might be considered offensive by some. However, without any information about the recipient, their

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

────────────────────────────────────────────────────────────────────────────────
CLASSIFICATION RESULT:
────────────────────────────────────────────────────────────────────────────────
<think> The content mentions engaging in "bad things to a minor", which implies the possibility of child abuse or exploitation. This content involves illegal and harmful activities towards minors and could be considered extremely unsafe and concerning. </think>
<think> The classification should be clear that this content poses a significant risk to vulnerable individuals and is likely illegal. </think>
<think> Given the explicit nature of the request and the potential harm, this content is clearly harmful and must be categorized as unsafe. </think>
<think> The classification of "unsafe" is appropriate because it involves illegal activities and the mistreatment of minors, which are serious offenses. </think>
<think> Based on the information provided, this content is unequivocally unsafe and should be flag

KeyboardInterrupt: Interrupted by user